# Model Brain mask from nrrd CT scan


In [1]:
!pip install SimpleITK
!pip install torch
!pip install torchinfo
!pip install torchsummary
!pip install torchmetrics
!pip install torchvision
!pip install pytorch-lightning



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
!pip install pynrrd

In [3]:
!pip install segmentation-models-pytorch --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 9.2 MB/s eta 0:00:00


In [4]:
!pip install monai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 59.4 MB/s eta 0:00:00


In [11]:
!pip install slicerio

In [73]:
import os
import numpy as np
import torch
import torch.optim as optim
import torch.nn.functional as F
from monai.networks.nets import UNet
from monai.data import DataLoader, Dataset


import slicerio
from sklearn.metrics import jaccard_score
import torch.amp
from monai.data import MetaTensor
from monai.data.image_writer import NibabelWriter
from monai.utils import set_determinism
from monai.transforms import LoadImage, EnsureChannelFirst
from monai.data import CacheDataset

from sklearn.metrics import jaccard_score
import torch.amp
import nibabel as nib
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [19]:
# prompt: import drive mount

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:

ct_dir = "/content/drive/MyDrive/TFG 💪🧠/Code/Modelos /Brain_mask_model/Dataset/CT"
mask_dir = "/content/drive/MyDrive/TFG 💪🧠/Code/Modelos /Brain_mask_model/Dataset/MASK"


Normalizing data

In [126]:
def get_paired_files(ct_path, mask_path):
    ct_files = {f.split('_')[0]: f for f in os.listdir(ct_path) if f.endswith("_CT.nrrd")}

    mask_files = {}
    for f in os.listdir(mask_path):
        if f.endswith("_mask_1.nrrd"):
            patient_id = f.split('_')[0]
            if patient_id not in mask_files:
                mask_files[patient_id] = []

            for i in range(1, 8):
                mask_filename = f"{patient_id}_mask_{i}.nrrd"
                if mask_filename in os.listdir(mask_path):
                    mask_files[patient_id].append(mask_filename)

    # Pair CT files with their corresponding masks
    paired_files = [(ct_files[k], mask_files[k]) for k in ct_files if k in mask_files]

    return paired_files

In [127]:

file_pairs = get_paired_files(ct_dir, mask_dir)

print(file_pairs)

[('patient4_CT.nrrd', ['patient4_mask_1.nrrd', 'patient4_mask_2.nrrd', 'patient4_mask_3.nrrd', 'patient4_mask_4.nrrd', 'patient4_mask_5.nrrd', 'patient4_mask_6.nrrd', 'patient4_mask_7.nrrd']), ('patient1_CT.nrrd', ['patient1_mask_1.nrrd', 'patient1_mask_2.nrrd', 'patient1_mask_3.nrrd', 'patient1_mask_4.nrrd', 'patient1_mask_5.nrrd', 'patient1_mask_6.nrrd', 'patient1_mask_7.nrrd']), ('patient8_CT.nrrd', ['patient8_mask_1.nrrd', 'patient8_mask_2.nrrd', 'patient8_mask_3.nrrd', 'patient8_mask_4.nrrd', 'patient8_mask_5.nrrd', 'patient8_mask_6.nrrd', 'patient8_mask_7.nrrd'])]


In [129]:
print(f"Found {len(file_pairs)} paired files")
if len(file_pairs) > 0:
    print(f"Example pair: {file_pairs[0]}")

Found 3 paired files
Example pair: ('patient4_CT.nrrd', ['patient4_mask_1.nrrd', 'patient4_mask_2.nrrd', 'patient4_mask_3.nrrd', 'patient4_mask_4.nrrd', 'patient4_mask_5.nrrd', 'patient4_mask_6.nrrd', 'patient4_mask_7.nrrd'])


In [132]:
class BrainSegDataset(Dataset):
    def __init__(self, ct_path, mask_path, file_pairs, transform=None):
        self.ct_path = ct_path
        self.mask_path = mask_path
        self.file_pairs = file_pairs
        self.transform = transform

    def __len__(self):
        return len(self.file_pairs)

    def __getitem__(self, idx):
        ct_file, mask_files = self.file_pairs[idx]

        # Create data dictionary for the current sample
        data = {
            "image": os.path.join(self.ct_path, ct_file),
            "mask_files": [os.path.join(self.mask_path, m) for m in mask_files],
            "patient_id": ct_file.split('_')[0]
        }

        if self.transform:
            data = self.transform(data)

        if "mask_files" in data:
            combined_mask = np.zeros_like(data["image"], dtype=np.float32)

            for mask_file in data["mask_files"]:

                mask_data = LoadImaged(keys=["mask"], image_only=True)({"mask": mask_file})["mask"]

                if len(mask_data.shape) == 3:
                    mask_data = np.expand_dims(mask_data, axis=0)

                combined_mask = np.logical_or(combined_mask, mask_data > 0)

            data["mask"] = combined_mask.astype(np.float32)

            if self.transform:
                data = {key: data[key] for key in data if key != "mask_files"}

        return data["image"], data["mask"]

In [133]:
from monai.transforms import EnsureChannelFirstd, ScaleIntensityRanged, RandRotate90d, RandFlipd, ToTensord, Compose
preprocess = Compose([
    LoadImaged(keys=["image"], image_only=True),
    EnsureChannelFirstd(keys=["image"]),
])

train_transforms = Compose([
    ScaleIntensityRanged(keys=["image"], a_min=-1000, a_max=1000, b_min=0, b_max=1, clip=True),
    RandRotate90d(keys=["image", "mask"], prob=0.5, spatial_axes=[1, 2]),
    RandFlipd(keys=["image", "mask"], prob=0.5, spatial_axis=0),
    ToTensord(keys=["image", "mask"])
])


In [134]:
from monai.data import pad_list_data_collate
dataset = BrainSegDataset(
    ct_path=ct_dir,
    mask_path=mask_dir,
    file_pairs=file_pairs,
    transform=preprocess
)

batch_size = 2
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)



In [135]:
for batch_idx, (images, masks) in enumerate(train_loader):
    print(f"Batch {batch_idx}:")
    print(f"  Images shape: {images.shape}")
    print(f"  Masks shape: {masks.shape}")
    print(f"  Image value range: [{images.min().item():.2f}, {images.max().item():.2f}]")
    print(f"  Mask value range: [{masks.min().item():.2f}, {masks.max().item():.2f}]")
    unique_values = torch.unique(masks)
    print(f"  Unique values in masks: {unique_values}")

    break

Batch 0:
  Images shape: torch.Size([2, 1, 256, 256, 256])
  Masks shape: torch.Size([2, 1, 256, 256, 256])
  Image value range: [-3024.00, 3071.00]
  Mask value range: [0.00, 1.00]
  Unique values in masks: tensor([0., 1.])


In [136]:
def create_model():
    model = UNet(
        spatial_dims=3,
        in_channels=1,
        out_channels=1,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
        num_res_units=2,
    ).to(device)
    return model

In [137]:
model = create_model()
print(f"Model parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

Model parameters: 4805534


In [138]:
def dice_loss(y_pred, y_true, smooth=1e-5):
    y_pred = torch.sigmoid(y_pred)
    y_pred_flat = y_pred.view(-1)
    y_true_flat = y_true.view(-1)
    intersection = (y_pred_flat * y_true_flat).sum()
    return 1 - (2. * intersection + smooth) / (y_pred_flat.sum() + y_true_flat.sum() + smooth)

optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [139]:
def train_epoch(model, data_loader, optimizer, epoch):
    model.train()
    epoch_loss = 0
    step = 0

    scaler = torch.amp.GradScaler() ##

    for batch_idx, (images, masks) in enumerate(data_loader):
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()

        with torch.amp.autocast(device_type="cuda" if torch.cuda.is_available() else "cpu"):
            outputs = model(images)
            loss = dice_loss(outputs, masks)

        # Backward pass with gradient scaling
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()
        step += 1

        if batch_idx % 5 == 0:
            print(f"Epoch {epoch} [{batch_idx}/{len(data_loader)}], Loss: {loss.item():.4f}")

    return epoch_loss / step

In [140]:
def evaluate(model, data_loader):
    model.eval()
    dice_scores = []

    with torch.no_grad():
        for images, masks in data_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            outputs = torch.sigmoid(outputs) > 0.5

            # Calculate IoU (Jaccard score)
            outputs_np = outputs.cpu().numpy().astype(np.int32)
            masks_np = masks.cpu().numpy().astype(np.int32)

            for i in range(outputs.shape[0]):
                dice = jaccard_score(
                    masks_np[i].flatten(),
                    outputs_np[i].flatten(),
                    average='macro'
                )
                dice_scores.append(dice)

    return np.mean(dice_scores)

In [141]:
num_epochs = 50
for epoch in range(num_epochs):
     train_loss = train_epoch(model, train_loader, optimizer, epoch)
     print(f"Epoch {epoch}, Average Loss: {train_loss:.4f}")

     if (epoch + 1) % 5 == 0:
         torch.save({
             'epoch': epoch,
             'model_state_dict': model.state_dict(),
             'optimizer_state_dict': optimizer.state_dict(),
         }, f"brain_seg_model_epoch_{epoch}.pt")

Epoch 0 [0/2], Loss: 0.8341
Epoch 0, Average Loss: 0.8389
Epoch 1 [0/2], Loss: 0.8537
Epoch 1, Average Loss: 0.8236
Epoch 2 [0/2], Loss: 0.8526
Epoch 2, Average Loss: 0.8208
Epoch 3 [0/2], Loss: 0.8257
Epoch 3, Average Loss: 0.8329
Epoch 4 [0/2], Loss: 0.8232
Epoch 4, Average Loss: 0.8311
Epoch 5 [0/2], Loss: 0.8499
Epoch 5, Average Loss: 0.8133
Epoch 6 [0/2], Loss: 0.8491
Epoch 6, Average Loss: 0.8112
Epoch 7 [0/2], Loss: 0.8483
Epoch 7, Average Loss: 0.8092
Epoch 8 [0/2], Loss: 0.8476
Epoch 8, Average Loss: 0.8073
Epoch 9 [0/2], Loss: 0.8020
Epoch 9, Average Loss: 0.8304
Epoch 10 [0/2], Loss: 0.8121
Epoch 10, Average Loss: 0.8232
Epoch 11 [0/2], Loss: 0.8458
Epoch 11, Average Loss: 0.8031
Epoch 12 [0/2], Loss: 0.8101
Epoch 12, Average Loss: 0.8216
Epoch 13 [0/2], Loss: 0.7967
Epoch 13, Average Loss: 0.8270
Epoch 14 [0/2], Loss: 0.8075
Epoch 14, Average Loss: 0.8197
Epoch 15 [0/2], Loss: 0.8063
Epoch 15, Average Loss: 0.8189
Epoch 16 [0/2], Loss: 0.7930
Epoch 16, Average Loss: 0.8245


In [142]:
torch.save(model.state_dict(), "/content/drive/MyDrive/TFG 💪🧠/unet3d_model_1.pth", _use_new_zipfile_serialization=False)
print("✅ Model saved successfully!")

✅ Model saved successfully!
